In [1]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [2]:
lstm = nn.LSTM(3, 3) # Input dim is 3, output dim is 3
inputs = [autograd.Variable(torch.randn((1, 3)))
             for _ in range(5)]

hidden = (autograd.Variable(torch.randn(1, 1, 3)),
          autograd.Variable(torch.randn((1, 3, 3))))

for i in inputs:
    
    out, hidden = lstm(i.view(1, 1, -1), hidden)
    

In [3]:
inputs = torch.cat(inputs).view(len(inputs), 1, -1)
hidden = (autograd.Variable(torch.randn(1, 1, 3)), 
          autograd.Variable(torch.randn((1, 1, 3))))
out, hidden = lstm(inputs, hidden)
print(out)
print(hidden)

Variable containing:
(0 ,.,.) = 
 -0.2945 -0.3090  0.0366

(1 ,.,.) = 
 -0.5580 -0.1228  0.0714

(2 ,.,.) = 
 -0.4122 -0.0834  0.0380

(3 ,.,.) = 
 -0.1954 -0.0010  0.0192

(4 ,.,.) = 
 -0.3722  0.0672  0.1393
[torch.FloatTensor of size 5x1x3]

(Variable containing:
(0 ,.,.) = 
 -0.3722  0.0672  0.1393
[torch.FloatTensor of size 1x1x3]
, Variable containing:
(0 ,.,.) = 
 -1.2037  0.3431  0.1663
[torch.FloatTensor of size 1x1x3]
)


In [4]:
torch.randn(1,1, 3)


(0 ,.,.) = 
  0.0554  1.1340 -0.5326
[torch.FloatTensor of size 1x1x3]

# Example: An LSTM for Part-of-Speech Tagging

In [5]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    tensor = torch.LongTensor(idxs)
    return autograd.Variable(tensor)

training_data = [
    ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])
]
word_to_ix = {}
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)

print(word_to_ix)
tag_to_ix = {"DET":0, "NN":1, "V":2}

EMBEDDING_DIM = 6
HIDDEN_DIM = 6

{'The': 0, 'dog': 1, 'ate': 2, 'the': 3, 'apple': 4, 'Everybody': 5, 'read': 6, 'that': 7, 'book': 8}


In [6]:
class LSTMTagger(nn.Module):
    
    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim
        
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        self.hidden = self.init_hidden()
        
    def init_hidden(self):
        return (autograd.Variable(torch.zeros(1, 1, self.hidden_dim)),
                autograd.Variable(torch.zeros(1, 1, self.hidden_dim)))
    
    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, self.hidden = self.lstm(embeds.view(len(sentence), 1, -1), self.hidden)
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [10]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

inputs = prepare_sequence(training_data[0][0], word_to_ix)
tag_scores = model(inputs)
print(tag_scores)

for epoch in range(300):
    for sentence, tags in training_data:
        model.zero_grad()
        
        model.hidden = model.init_hidden()
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)
        
        tag_scores = model(sentence_in)
        
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

inputs = prepare_sequence(training_data[0][0], word_to_ix)
tag_scores = model(inputs)

print(tag_scores)

Variable containing:
-1.3746 -1.0343 -0.9375
-1.3714 -1.0030 -0.9690
-1.3815 -1.0487 -0.9203
-1.3029 -1.0200 -1.0005
-1.2731 -1.0083 -1.0351
[torch.FloatTensor of size 5x3]

Variable containing:
-0.1990 -1.9759 -3.1737
-4.5696 -0.0131 -5.9220
-3.2873 -4.9873 -0.0452
-0.0348 -4.2219 -3.9370
-4.0370 -0.0197 -6.2944
[torch.FloatTensor of size 5x3]

